<a href="https://colab.research.google.com/github/Mahadevan0507/Credit-Card-Fraud-Detection/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'creditcardfraud:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F310%2F23498%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240808%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240808T151023Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4609324a23494c40642cb1efe580ae58bebe18751074a6d4e65dd0b7f27163ba84291785da32e52b45e58cf3a343aaf7e4ddf7dfc6e45d86f9912016040d2c8f85af960885f2edbbf007a8e3f005abe653305f36ccc116228421c3fb0641e9a5aae943fbf0723c7fe50c987af59ab66acda14cdf1e4ba67a1eda34fc392544e7b5500cea708eac7277ef945f677cdf2dfd8a52e4c29a141eb74565d25f1d1eea8f24bbf8dde5a17fbdf26d2abe6507651ea0f5f7c4cbb05aac1b995ab3c0f0ee0c423a0e8a4ab3fc42874ce275d8f610daa8495817fed30c2f2b5999520eefc7748026ab0ee8329ee02d196cf5ffda34cb82217ec62d0240fba008a36824ac68,credit-card-fraud-detection/pytorch/default/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F75714%2F90321%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240808%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240808T151023Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D330a0bbe709e18208fdefef2f1d2e4291af3ac10a9a294275d02da7132a4474983f26e6e5ce66a0cf9af0466c1d3c8e635b77bf75de32322bb4c989aabb74fb1e57f94cd67b3b74ab130774c93c86ec8756d185bf01f395de57bea3b416252082008ea5eaa408f5c5c4754e2279afe43d22dd34a7ab3da9015873a057f1b802402fa2f7894661afa63478584462abf1514728409431c0031b730298f7e4e39eaa9a7b4739e78115cbbf27c5a3e8f9eb1b3f2b1abb169fd69b18396c84f30e2942b92a9cf80d64b0369958725e46ee1edadcabda6cd8e99fb53e83e923e5233c01b38fe763dc9787442f97169d25227de5f0b0b3bbd3ca7f142dac0573fb85a21'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 69155672 bytes downloaded
Downloaded and uncompressed: creditcardfraud
[==================================================] 15429 bytes downloaded
Downloaded and uncompressed: credit-card-fraud-detection/pytorch/default/1
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card-fraud-detection/pytorch/default/1/credit-card-fraud-detection.ipynb
/kaggle/input/creditcardfraud/creditcard.csv


In [3]:
df=pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

In [4]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [7]:
df.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [8]:
df.dropna(inplace=True)

In [9]:
df['Class'].value_counts()

,count
Class,
0,284315
1,492


In [10]:
not_fraud = df[df.Class == 0]
fraud = df[df.Class == 1]

In [11]:
not_fraud.shape

(284315, 31)

In [12]:
fraud.shape

(492, 31)

In [13]:
not_fraud.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,...,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.000000,284315.0
mean,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022,0.0
std,47484.015786,1.929814,1.636146,1.459429,1.399333,1.356952,1.329913,1.178812,1.161283,1.089372,...,0.716743,0.723668,0.621541,0.605776,0.520673,0.482241,0.399847,0.329570,250.105092,0.0
min,0.000000,-56.407510,-72.715728,-48.325589,-5.683171,-113.743307,-26.160506,-31.764946,-73.216718,-6.290730,...,-34.830382,-10.933144,-44.807735,-2.836627,-10.295397,-2.604551,-22.565679,-15.430084,0.000000,0.0
25%,54230.000000,-0.917544,-0.599473,-0.884541,-0.850077,-0.689398,-0.766847,-0.551442,-0.208633,-0.640412,...,-0.228509,-0.542403,-0.161702,-0.354425,-0.317145,-0.327074,-0.070852,-0.052950,5.650000,0.0
50%,84711.000000,0.020023,0.064070,0.182158,-0.022405,-0.053457,-0.273123,0.041138,0.022041,-0.049964,...,-0.029821,0.006736,-0.011147,0.041082,0.016417,-0.052227,0.001230,0.011199,22.000000,0.0
75%,139333.000000,1.316218,0.800446,1.028372,0.737624,0.612181,0.399619,0.571019,0.326200,0.598230,...,0.185626,0.528407,0.147522,0.439869,0.350594,0.240671,0.090573,0.077962,77.050000,0.0
max,172792.000000,2.454930,18.902453,9.382558,16.875344,34.801666,73.301626,120.589494,18.709255,15.594995,...,22.614889,10.503090,22.528412,4.584549,7.519589,3.517346,31.612198,33.847808,25691.160000,0.0


In [14]:
fraud.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,...,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.0
mean,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321,1.0
std,47835.365138,6.783687,4.291216,7.110937,2.873318,5.372468,1.858124,7.206773,6.797831,2.500896,...,3.869304,1.494602,1.579642,0.515577,0.797205,0.471679,1.376766,0.547291,256.683288,0.0
min,406.000000,-30.552380,-8.402154,-31.103685,-1.313275,-22.105532,-6.406267,-43.557242,-41.044261,-13.434066,...,-22.797604,-8.887017,-19.254328,-2.028024,-4.781606,-1.152671,-7.263482,-1.869290,0.000000,1.0
25%,41241.500000,-6.036063,1.188226,-8.643489,2.373050,-4.792835,-2.501511,-7.965295,-0.195336,-3.872383,...,0.041787,-0.533764,-0.342175,-0.436809,-0.314348,-0.259416,-0.020025,-0.108868,1.000000,1.0
50%,75568.500000,-2.342497,2.717869,-5.075257,4.177147,-1.522962,-1.424616,-3.034402,0.621508,-2.208768,...,0.592146,0.048434,-0.073135,-0.060795,0.088371,0.004321,0.394926,0.146344,9.250000,1.0
75%,128483.000000,-0.419200,4.971257,-2.276185,6.348729,0.214562,-0.413216,-0.945954,1.764879,-0.787850,...,1.244611,0.617474,0.308378,0.285328,0.456515,0.396733,0.826029,0.381152,105.890000,1.0
max,170348.000000,2.132386,22.057729,2.250210,12.114672,11.095089,6.474115,5.802537,20.007208,3.353525,...,27.202839,8.361985,5.466230,1.091435,2.208209,2.745261,3.052358,1.779364,2125.870000,1.0


In [15]:
not_fraud_sample=not_fraud.sample(n=492)

In [16]:
not_fraud_sample

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
277416,167642.0,2.052133,0.059843,-1.817728,0.237598,0.568886,-0.399829,0.039718,-0.039706,0.336821,...,-0.338494,-0.931293,0.297109,0.170919,-0.267185,0.174342,-0.070495,-0.044760,1.98,0
170350,120140.0,1.826419,-0.106954,-2.469643,1.014312,1.074387,-0.584229,0.994583,-0.418222,-0.471782,...,0.250467,0.510140,-0.334761,-1.006710,0.630793,-0.388734,-0.067446,-0.060996,150.00,0
242385,151472.0,-0.934554,-0.049423,0.351146,-2.274465,3.002017,3.600342,-0.232126,1.174259,0.074546,...,-0.121399,-0.638322,0.036269,0.663631,-0.197179,-0.173324,0.064878,0.125443,9.99,0
144680,86321.0,-1.114728,0.603886,1.754698,-1.402008,-0.079864,0.378075,0.320684,0.188861,0.905338,...,0.008564,0.654152,-0.298714,-0.210417,-0.137461,1.008257,0.388602,0.036333,30.43,0
87532,61726.0,-0.135881,0.409301,2.047115,1.495652,-0.142386,1.435903,0.098145,0.240502,0.617429,...,-0.331838,-0.331127,-0.020104,-0.290880,-0.346038,-0.480361,-0.139405,-0.211640,27.90,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201125,133739.0,2.362906,-0.509636,-2.883951,-1.113263,0.648281,-1.133065,0.361738,-0.521671,-0.859546,...,0.409676,1.195349,-0.271797,0.106097,0.849328,0.314003,-0.107739,-0.096071,10.00,0
275478,166551.0,1.973448,-0.576971,-0.688902,-0.006955,-0.282661,0.325199,-0.737734,0.252151,1.031892,...,-0.213621,-0.670061,0.363582,0.054641,-0.609661,0.261032,-0.055230,-0.051253,27.50,0
101935,67987.0,-1.329170,1.442239,0.995968,0.879001,-0.583026,-1.022531,0.952195,-0.247203,0.186299,...,0.064353,0.466005,0.024497,0.899620,-0.202627,-0.369150,0.240144,0.429098,63.42,0
205171,135621.0,0.083060,0.993025,-0.330925,-0.637560,0.956486,-0.490678,0.882855,0.039971,-0.353825,...,-0.272288,-0.657085,0.106318,0.607465,-0.473560,0.091475,0.222769,0.075513,4.99,0


In [17]:
new_df=pd.concat([not_fraud_sample,fraud] , axis=0)

In [18]:
new_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
277416,167642.0,2.052133,0.059843,-1.817728,0.237598,0.568886,-0.399829,0.039718,-0.039706,0.336821,...,-0.338494,-0.931293,0.297109,0.170919,-0.267185,0.174342,-0.070495,-0.044760,1.98,0
170350,120140.0,1.826419,-0.106954,-2.469643,1.014312,1.074387,-0.584229,0.994583,-0.418222,-0.471782,...,0.250467,0.510140,-0.334761,-1.006710,0.630793,-0.388734,-0.067446,-0.060996,150.00,0
242385,151472.0,-0.934554,-0.049423,0.351146,-2.274465,3.002017,3.600342,-0.232126,1.174259,0.074546,...,-0.121399,-0.638322,0.036269,0.663631,-0.197179,-0.173324,0.064878,0.125443,9.99,0
144680,86321.0,-1.114728,0.603886,1.754698,-1.402008,-0.079864,0.378075,0.320684,0.188861,0.905338,...,0.008564,0.654152,-0.298714,-0.210417,-0.137461,1.008257,0.388602,0.036333,30.43,0
87532,61726.0,-0.135881,0.409301,2.047115,1.495652,-0.142386,1.435903,0.098145,0.240502,0.617429,...,-0.331838,-0.331127,-0.020104,-0.290880,-0.346038,-0.480361,-0.139405,-0.211640,27.90,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [19]:
new_df['Class'].value_counts()

,count
Class,
0,492
1,492


In [20]:
new_df.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,98818.099593,-0.022231,0.192627,-0.050470,0.104583,0.029152,-0.017213,0.037931,-0.085299,-0.039607,...,0.032698,-0.048501,0.006577,0.014941,0.010805,0.023810,0.000080,-0.000664,0.001368,77.335549
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [21]:
from sklearn.model_selection import train_test_split

X = new_df.drop('Class', axis=1)
y = new_df['Class']



In [22]:
X

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
277416,167642.0,2.052133,0.059843,-1.817728,0.237598,0.568886,-0.399829,0.039718,-0.039706,0.336821,...,-0.169649,-0.338494,-0.931293,0.297109,0.170919,-0.267185,0.174342,-0.070495,-0.044760,1.98
170350,120140.0,1.826419,-0.106954,-2.469643,1.014312,1.074387,-0.584229,0.994583,-0.418222,-0.471782,...,0.077500,0.250467,0.510140,-0.334761,-1.006710,0.630793,-0.388734,-0.067446,-0.060996,150.00
242385,151472.0,-0.934554,-0.049423,0.351146,-2.274465,3.002017,3.600342,-0.232126,1.174259,0.074546,...,-0.065177,-0.121399,-0.638322,0.036269,0.663631,-0.197179,-0.173324,0.064878,0.125443,9.99
144680,86321.0,-1.114728,0.603886,1.754698,-1.402008,-0.079864,0.378075,0.320684,0.188861,0.905338,...,0.265838,0.008564,0.654152,-0.298714,-0.210417,-0.137461,1.008257,0.388602,0.036333,30.43
87532,61726.0,-0.135881,0.409301,2.047115,1.495652,-0.142386,1.435903,0.098145,0.240502,0.617429,...,0.024099,-0.331838,-0.331127,-0.020104,-0.290880,-0.346038,-0.480361,-0.139405,-0.211640,27.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00


In [23]:
y

,Class
277416,0
170350,0
242385,0
144680,0
87532,0
...,...
279863,1
280143,1
280149,1
281144,1


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y , random_state=2 )

In [25]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [26]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [27]:
y_pred = logreg.predict(X_test)


In [28]:
X_train_prediction=logreg.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,y_train)

In [29]:
training_data_accuracy

0.9377382465057179

In [30]:

X_test_prediction = logreg.predict(X_test)

test_data_accuracy = accuracy_score(X_test_prediction,y_test)

print('Accuracy score on Test Data: ', test_data_accuracy)

Accuracy score on Test Data:  0.9137055837563451
